In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import numpy as np
import scipy.sparse as sps
from DataReader import DataReaderChallangePolimi2017
dr = DataReaderChallangePolimi2017(evaluation = True, rebuild = False)

urm = dr.getURM_csr()
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
test = dr.getTestURM_csr()
ucm = dr.getOwnersUCM_csr()

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [9]:
urm_plus = sps.vstack([urm,icm.T]).tocsr()

In [6]:
ucm_plus = sps.vstack([urm.T,ucm.T]).tocsr()

In [12]:
print urm_plus.shape
print ucm_plus.shape

(134601, 100000)
(145170, 57561)


In [13]:
recommender = SLIM_BPR_Cython(urm_plus, recompile_cython=False, positive_threshold=1, sparse_weights=True )
logFile = open("SLIM_BPR_Cython2.txt", "a")
recommender.fit(epochs=10, validate_every_N_epochs= 1000, URM_test=test,topK=100,
                logFile=logFile, batch_size=1, sgd_mode='adagrad', learning_rate=1e-4,
                 start_validation_after_N_epochs = 1000,
                 lambda_i = 0.0025, lambda_j = 0.0025,  # lambda_i = 0.0025, lambda_j = 0.00025,
                )

Epoch 0 of 10 complete in 0.00 minutes
Processed 500000 ( 48.94% ) in 5.53 seconds. Sample per second: 90367
Processed 1000000 ( 97.89% ) in 6.60 seconds. Sample per second: 86237
Processed 1021573 ( 100.00% ) in 0.87 seconds. Sample per second: 86042
Return S matrix to python caller
Epoch 1 of 10 complete in 0.24 minutes
Processed 500000 ( 48.94% ) in 5.73 seconds. Sample per second: 87199
Processed 1000000 ( 97.89% ) in 6.72 seconds. Sample per second: 85302
Processed 1021573 ( 100.00% ) in 1.01 seconds. Sample per second: 85082
Return S matrix to python caller
Epoch 2 of 10 complete in 0.26 minutes
Processed 500000 ( 48.94% ) in 5.48 seconds. Sample per second: 91308
Processed 1000000 ( 97.89% ) in 6.48 seconds. Sample per second: 87108
Processed 1021573 ( 100.00% ) in 0.77 seconds. Sample per second: 86802
Return S matrix to python caller
Epoch 3 of 10 complete in 0.26 minutes
Processed 500000 ( 48.94% ) in 6.23 seconds. Sample per second: 80283
Processed 1000000 ( 97.89% ) in 6.34

In [14]:
sim = recommender.getSimilarity()
#dr.evaluateMAP(urm*sim) #urmplus
#dr.evaluateMAP(sim*urm) #ucmplus

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0371


0.037139972222222289

In [10]:
dr.save_sparse_csr(array=sim,filename="./DumpData/sim_bpr_10e_100k_fulltrain")